# Aprendizaje de reglas de asociación

trabajo práctico:
Obtener reglas de asociación entre películas en el dataset movielens, al estilo de: https://rpubs.com/vitidN/203264 pero usando la notebook de reglas de asociación de Valeria .
Aplicar diferentes métricas para ordenamiento de las reglas Hacer un pequeño informe (entre 200 y 500 palabras, puede ser más pero no mucho más) de interpretación de los efectos de las métricas aplicadas

In [1]:
import numpy as np
from itertools import combinations, groupby
from collections import Counter
import pandas as pd
import sys
from IPython.display import display

In [2]:
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))


# Utilizando la base reducida
http://files.grouplens.org/datasets/movielens/

ml-latest-small

In [3]:
#ratings y ID de usuario
ratings = pd.read_csv('C://Juliana/Diplomatura/05-Aprendizaje_NoSupervisado/AprendizajeNOSupervisado-master/ml-latest-small/ratings.csv')
print('ratings -- dimensions: {0};   size: {1}'.format(ratings.shape, size(ratings)))
display(ratings.head())





ratings -- dimensions: (100836, 4);   size: 3.23 MB


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
items_names = pd.read_csv('C://Juliana/Diplomatura/05-Aprendizaje_NoSupervisado/AprendizajeNOSupervisado-master/ml-latest-small/movies.csv')
display(items_names.head())
print('items_names -- dimensions: {0};   size: {1}'.format(items_names.shape, size(items_names)))

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


items_names -- dimensions: (9742, 3);   size: 1.60 MB


In [5]:
#unir los datos de rating con los de los titulos de las peliculas
pelis_df = pd.merge(ratings[['userId','movieId']], items_names[['movieId','title']] ,on='movieId', how= "inner")

display(pelis_df.head())
pelis_df=pelis_df.sort_values( by='userId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
pelis_df=pelis_df.values[:,[0,2]]
print(pelis_df)

,userId,movieId,title
0,1,1,Toy Story (1995)
1,5,1,Toy Story (1995)
2,7,1,Toy Story (1995)
3,15,1,Toy Story (1995)
4,17,1,Toy Story (1995)


[[1 'Toy Story (1995)']
 [1 'Psycho (1998)']
 [1 'Rushmore (1998)']
 ...
 [610 'Coraline (2009)']
 [610 'Serious Man, A (2009)']
 [610 '31 (2016)']]


In [6]:
#genero transacciones
transactions=[]
for orders_id, order_object in groupby(pelis_df, lambda x: x[0]):
    transactions.append([item[1] for item in order_object])
#print(transactions)

In [58]:
from efficient_apriori import apriori
#itemsets, rules = apriori(transactions, min_support=0.1, min_confidence=0.3)

#min_support y min_confiden deben estar entre 0 y 1 
#uso el apriori
itemsets, rules = apriori(transactions, min_support=0.1,  min_confidence=0.5, max_length=2)

In [59]:
 
print(len(rules))

5376


In [60]:
rules=sorted(rules, key=lambda rule: rule.confidence)
for rule in rules:
  print(rule)

{Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)} -> {Alien (1979)} (conf: 0.500, supp: 0.164, lift: 2.089, conv: 1.521)
{Aliens (1986)} -> {True Lies (1994)} (conf: 0.500, supp: 0.103, lift: 1.713, conv: 1.416)
{American Beauty (1999)} -> {Gladiator (2000)} (conf: 0.500, supp: 0.167, lift: 1.794, conv: 1.443)
{Monty Python and the Holy Grail (1975)} -> {Apocalypse Now (1979)} (conf: 0.500, supp: 0.111, lift: 2.850, conv: 1.649)
{Babe (1995)} -> {Matrix, The (1999)} (conf: 0.500, supp: 0.105, lift: 1.097, conv: 1.089)
{Babe (1995)} -> {Twelve Monkeys (a.k.a. 12 Monkeys) (1995)} (conf: 0.500, supp: 0.105, lift: 1.723, conv: 1.420)
{Saving Private Ryan (1998)} -> {Back to the Future (1985)} (conf: 0.500, supp: 0.154, lift: 1.784, conv: 1.439)
{Mrs. Doubtfire (1993)} -> {Batman Forever (1995)} (conf: 0.500, supp: 0.118, lift: 2.226, conv: 1.551)
{Beauty and the Beast (1991)} -> {Pretty Woman (1990)} (conf: 0.500, supp: 0.120, lift: 2.259, conv: 1.557)
{Blade

{Die Hard: With a Vengeance (1995)} -> {Mrs. Doubtfire (1993)} (conf: 0.521, supp: 0.123, lift: 2.206, conv: 1.594)
{Die Hard: With a Vengeance (1995)} -> {Waterworld (1995)} (conf: 0.521, supp: 0.123, lift: 2.763, conv: 1.694)
{Godfather, The (1972)} -> {Sixth Sense, The (1999)} (conf: 0.521, supp: 0.164, lift: 1.775, conv: 1.475)
{Godfather, The (1972)} -> {Terminator 2: Judgment Day (1991)} (conf: 0.521, supp: 0.164, lift: 1.418, conv: 1.321)
{Mrs. Doubtfire (1993)} -> {Sleepless in Seattle (1993)} (conf: 0.521, supp: 0.123, lift: 2.997, conv: 1.724)
{Mrs. Doubtfire (1993)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.521, supp: 0.123, lift: 1.266, conv: 1.228)
{Willy Wonka & the Chocolate Factory (1971)} -> {Back to the Future (1985)} (conf: 0.521, supp: 0.102, lift: 1.859, conv: 1.502)
{Ocean's Eleven (2001)} -> {Batman Begins (2005)} (conf: 0.521, supp: 0.102, lift: 2.740, conv: 1.691)
{Willy Wonka & the Chocolate Factory (1971)} -> {Groundhog Day (1993)} (conf: 0.521,

{Gladiator (2000)} -> {Godfather, The (1972)} (conf: 0.541, supp: 0.151, lift: 1.719, conv: 1.493)
{Shrek (2001)} -> {Kill Bill: Vol. 1 (2003)} (conf: 0.541, supp: 0.151, lift: 2.520, conv: 1.711)
{Shrek (2001)} -> {Memento (2000)} (conf: 0.541, supp: 0.151, lift: 2.076, conv: 1.611)
{Fight Club (1999)} -> {Gladiator (2000)} (conf: 0.541, supp: 0.193, lift: 1.942, conv: 1.572)
{Fight Club (1999)} -> {Saving Private Ryan (1998)} (conf: 0.541, supp: 0.193, lift: 1.756, conv: 1.508)
{Fight Club (1999)} -> {Schindler's List (1993)} (conf: 0.541, supp: 0.193, lift: 1.501, conv: 1.394)
{Fight Club (1999)} -> {Seven (a.k.a. Se7en) (1995)} (conf: 0.541, supp: 0.193, lift: 1.627, conv: 1.455)
{X-Men (2000)} -> {Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)} (conf: 0.541, supp: 0.118, lift: 3.002, conv: 1.787)
{Dumb & Dumber (Dumb and Dumber) (1994)} -> {Toy Story (1995)} (conf: 0.541, supp: 0.118, lift: 1.536, conv: 1.412)
{X-Men (2000)} -> {Groundhog Day (1993)} (conf: 0.541, supp: 0

{Goodfellas (1990)} -> {Memento (2000)} (conf: 0.563, supp: 0.116, lift: 2.162, conv: 1.694)
{Pulp Fiction (1994)} -> {Jurassic Park (1993)} (conf: 0.564, supp: 0.284, lift: 1.444, conv: 1.397)
{Fargo (1996)} -> {Independence Day (a.k.a. ID4) (1996)} (conf: 0.564, supp: 0.167, lift: 1.702, conv: 1.532)
{Fargo (1996)} -> {Terminator 2: Judgment Day (1991)} (conf: 0.564, supp: 0.167, lift: 1.535, conv: 1.450)
{Jumanji (1995)} -> {Ace Ventura: Pet Detective (1994)} (conf: 0.564, supp: 0.102, lift: 2.136, conv: 1.687)
{Jumanji (1995)} -> {Back to the Future (1985)} (conf: 0.564, supp: 0.102, lift: 2.011, conv: 1.649)
{Kill Bill: Vol. 2 (2004)} -> {Braveheart (1995)} (conf: 0.564, supp: 0.102, lift: 1.451, conv: 1.401)
{Kill Bill: Vol. 2 (2004)} -> {Departed, The (2006)} (conf: 0.564, supp: 0.102, lift: 3.213, conv: 1.890)
{Jumanji (1995)} -> {Fugitive, The (1993)} (conf: 0.564, supp: 0.102, lift: 1.810, conv: 1.578)
{Men in Black (a.k.a. MIB) (1997)} -> {Groundhog Day (1993)} (conf: 0.564,

{Reservoir Dogs (1992)} -> {Godfather: Part II, The (1974)} (conf: 0.595, supp: 0.128, lift: 2.816, conv: 1.949)
{Terminator, The (1984)} -> {Indiana Jones and the Temple of Doom (1984)} (conf: 0.595, supp: 0.128, lift: 3.363, conv: 2.034)
{Reservoir Dogs (1992)} -> {Memento (2000)} (conf: 0.595, supp: 0.128, lift: 2.284, conv: 1.827)
{Saving Private Ryan (1998)} -> {American Beauty (1999)} (conf: 0.596, supp: 0.184, lift: 1.781, conv: 1.646)
{Good Will Hunting (1997)} -> {Gladiator (2000)} (conf: 0.596, supp: 0.138, lift: 2.138, conv: 1.784)
{Good Will Hunting (1997)} -> {Lord of the Rings: The Return of the King, The (2003)} (conf: 0.596, supp: 0.138, lift: 1.964, conv: 1.723)
{Good Will Hunting (1997)} -> {Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)} (conf: 0.596, supp: 0.138, lift: 1.817, conv: 1.663)
{Saving Private Ryan (1998)} -> {Schindler's List (1993)} (conf: 0.596, supp: 0.184, lift: 1.652, conv: 1.582)
{Alien (1979)} -> {Back to the Future

...

{V for Vendetta (2006)} -> {Seven (a.k.a. Se7en) (1995)} (conf: 0.640, supp: 0.105, lift: 1.923, conv: 1.853)
{V for Vendetta (2006)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.640, supp: 0.105, lift: 1.555, conv: 1.635)
{Matrix, The (1999)} -> {Pulp Fiction (1994)} (conf: 0.640, supp: 0.292, lift: 1.272, conv: 1.381)
{True Lies (1994)} -> {Aladdin (1992)} (conf: 0.640, supp: 0.187, lift: 2.135, conv: 1.947)
{Babe (1995)} -> {Braveheart (1995)} (conf: 0.641, supp: 0.134, lift: 1.649, conv: 1.701)
{American Pie (1999)} -> {Saving Private Ryan (1998)} (conf: 0.641, supp: 0.108, lift: 2.079, conv: 1.926)
{Crimson Tide (1995)} -> {Stargate (1994)} (conf: 0.641, supp: 0.108, lift: 2.792, conv: 2.145)
{Four Weddings and a Funeral (1994)} -> {Schindler's List (1993)} (conf: 0.641, supp: 0.108, lift: 1.777, conv: 1.780)
{Four Weddings and a Funeral (1994)} -> {Terminator 2: Judgment Day (1991)} (conf: 0.641, supp: 0.108, lift: 1.745, conv: 1.762)
{Princess Bride, The (1987)} -> {S

{Cast Away (2000)} -> {Catch Me If You Can (2002)} (conf: 0.680, supp: 0.111, lift: 3.607, conv: 2.536)
{Cast Away (2000)} -> {Lord of the Rings: The Two Towers, The (2002)} (conf: 0.680, supp: 0.111, lift: 2.206, conv: 2.162)
{V for Vendetta (2006)} -> {Dark Knight, The (2008)} (conf: 0.680, supp: 0.111, lift: 2.784, conv: 2.362)
{Truman Show, The (1998)} -> {Lord of the Rings: The Fellowship of the Ring, The (2001)} (conf: 0.680, supp: 0.139, lift: 2.095, conv: 2.111)
{Lion King, The (1994)} -> {Shawshank Redemption, The (1994)} (conf: 0.680, supp: 0.192, lift: 1.309, conv: 1.502)
{Spider-Man (2002)} -> {X-Men (2000)} (conf: 0.680, supp: 0.136, lift: 3.120, conv: 2.446)
{Avatar (2009)} -> {Iron Man (2008)} (conf: 0.680, supp: 0.108, lift: 4.415, conv: 2.647)
{Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)} -> {Fargo (1996)} (conf: 0.680, supp: 0.108, lift: 2.293, conv: 2.201)
{L.A. Confidential (1997)} -> {Godfather, The (1972)} (conf: 0.680, supp: 0.108,

{Jaws (1975)} -> {Independence Day (a.k.a. ID4) (1996)} (conf: 0.692, supp: 0.103, lift: 2.091, conv: 2.174)
{Jaws (1975)} -> {Indiana Jones and the Last Crusade (1989)} (conf: 0.692, supp: 0.103, lift: 3.016, conv: 2.504)
{Stand by Me (1986)} -> {Shawshank Redemption, The (1994)} (conf: 0.692, supp: 0.103, lift: 1.332, conv: 1.561)
{Star Trek: First Contact (1996)} -> {Star Wars: Episode V - The Empire Strikes Back (1980)} (conf: 0.692, supp: 0.103, lift: 2.001, conv: 2.126)
{Star Trek: First Contact (1996)} -> {Star Wars: Episode VI - Return of the Jedi (1983)} (conf: 0.692, supp: 0.103, lift: 2.155, conv: 2.206)
{Stargate (1994)} -> {Fugitive, The (1993)} (conf: 0.693, supp: 0.159, lift: 2.224, conv: 2.242)
{Indiana Jones and the Last Crusade (1989)} -> {Pulp Fiction (1994)} (conf: 0.693, supp: 0.159, lift: 1.377, conv: 1.617)
{Cliffhanger (1993)} -> {Dances with Wolves (1990)} (conf: 0.693, supp: 0.115, lift: 2.578, conv: 2.382)
{Firm, The (1993)} -> {Dances with Wolves (1990)} (co

{Stargate (1994)} -> {Braveheart (1995)} (conf: 0.750, supp: 0.172, lift: 1.930, conv: 2.446)
{Broken Arrow (1996)} -> {Independence Day (a.k.a. ID4) (1996)} (conf: 0.750, supp: 0.103, lift: 2.265, conv: 2.675)
{Bug's Life, A (1998)} -> {Toy Story (1995)} (conf: 0.750, supp: 0.113, lift: 2.128, conv: 2.590)
{Casablanca (1942)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.750, supp: 0.123, lift: 1.823, conv: 2.354)
{Clerks (1994)} -> {Forrest Gump (1994)} (conf: 0.750, supp: 0.128, lift: 1.391, conv: 1.843)
{Inglourious Basterds (2009)} -> {Dark Knight, The (2008)} (conf: 0.750, supp: 0.108, lift: 3.070, conv: 3.023)
{Die Hard: With a Vengeance (1995)} -> {True Lies (1994)} (conf: 0.750, supp: 0.177, lift: 2.570, conv: 2.833)
{Stargate (1994)} -> {Forrest Gump (1994)} (conf: 0.750, supp: 0.172, lift: 1.391, conv: 1.843)
{Home Alone (1990)} -> {Pulp Fiction (1994)} (conf: 0.750, supp: 0.143, lift: 1.490, conv: 1.987)
{Shrek 2 (2004)} -> {Incredibles, The (2004)} (conf: 0.750, 

{Harry Potter and the Prisoner of Azkaban (2004)} -> {Shrek (2001)} (conf: 0.796, supp: 0.121, lift: 2.855, conv: 3.531)
{Snatch (2000)} -> {Pulp Fiction (1994)} (conf: 0.796, supp: 0.121, lift: 1.581, conv: 2.431)
{Crimson Tide (1995)} -> {Batman (1989)} (conf: 0.796, supp: 0.134, lift: 2.569, conv: 3.385)
{Mask, The (1994)} -> {Jurassic Park (1993)} (conf: 0.796, supp: 0.205, lift: 2.041, conv: 2.992)
{Mission: Impossible (1996)} -> {Independence Day (a.k.a. ID4) (1996)} (conf: 0.796, supp: 0.211, lift: 2.405, conv: 3.283)
{Dumb & Dumber (Dumb and Dumber) (1994)} -> {Pulp Fiction (1994)} (conf: 0.797, supp: 0.174, lift: 1.584, conv: 2.447)
{Inception (2010)} -> {Matrix, The (1999)} (conf: 0.797, supp: 0.187, lift: 1.749, conv: 2.684)
{Dogma (1999)} -> {Forrest Gump (1994)} (conf: 0.797, supp: 0.103, lift: 1.479, conv: 2.274)
{Matrix Revolutions, The (2003)} -> {Forrest Gump (1994)} (conf: 0.797, supp: 0.103, lift: 1.479, conv: 2.274)
{Matrix Revolutions, The (2003)} -> {Star Wars: Ep

In [47]:
rules=sorted(rules, key=lambda rule: rule.support)
for rule in rules:
  print(rule)

{E.T. the Extra-Terrestrial (1982)} -> {Godfather: Part II, The (1974)} (conf: 0.500, supp: 0.100, lift: 2.364, conv: 1.577)
{E.T. the Extra-Terrestrial (1982)} -> {Good Will Hunting (1997)} (conf: 0.500, supp: 0.100, lift: 2.163, conv: 1.538)
{E.T. the Extra-Terrestrial (1982)} -> {Goodfellas (1990)} (conf: 0.500, supp: 0.100, lift: 2.421, conv: 1.587)
{E.T. the Extra-Terrestrial (1982)} -> {Lord of the Rings: The Two Towers, The (2002)} (conf: 0.500, supp: 0.100, lift: 1.622, conv: 1.384)
{E.T. the Extra-Terrestrial (1982)} -> {Rain Man (1988)} (conf: 0.500, supp: 0.100, lift: 3.177, conv: 1.685)
{E.T. the Extra-Terrestrial (1982)} -> {Who Framed Roger Rabbit? (1988)} (conf: 0.500, supp: 0.100, lift: 3.144, conv: 1.682)
{Spider-Man (2002)} -> {X2: X-Men United (2003)} (conf: 0.500, supp: 0.100, lift: 4.013, conv: 1.751)
{Austin Powers: The Spy Who Shagged Me (1999)} -> {Alien (1979)} (conf: 0.504, supp: 0.100, lift: 2.106, conv: 1.534)
{Austin Powers: The Spy Who Shagged Me (1999)} -

{Ghost (1990)} -> {Batman Forever (1995)} (conf: 0.557, supp: 0.105, lift: 2.478, conv: 1.748)
{Catch Me If You Can (2002)} -> {Godfather, The (1972)} (conf: 0.557, supp: 0.105, lift: 1.768, conv: 1.545)
{Catch Me If You Can (2002)} -> {Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)} (conf: 0.557, supp: 0.105, lift: 3.173, conv: 1.859)
{Catch Me If You Can (2002)} -> {Independence Day (a.k.a. ID4) (1996)} (conf: 0.557, supp: 0.105, lift: 1.681, conv: 1.508)
{Catch Me If You Can (2002)} -> {Matrix Reloaded, The (2003)} (conf: 0.557, supp: 0.105, lift: 3.536, conv: 1.900)
{Catch Me If You Can (2002)} -> {Usual Suspects, The (1995)} (conf: 0.557, supp: 0.105, lift: 1.664, conv: 1.501)
{Waterworld (1995)} -> {Twelve Monkeys (a.k.a. 12 Monkeys) (1995)} (conf: 0.557, supp: 0.105, lift: 1.918, conv: 1.601)
{Breakfast Club, The (1985)} -> {Die Hard (1988)} (conf: 0.566, supp: 0.105, lift: 2.383, conv: 1.758)
{Breakfast Club, The (1985)} -> {Star 

{X-Men (2000)} -> {E.T. the Extra-Terrestrial (1982)} (conf: 0.504, supp: 0.110, lift: 2.519, conv: 1.612)
{Léon: The Professional (a.k.a. The Professional) (Léon) (1994)} -> {Fugitive, The (1993)} (conf: 0.504, supp: 0.110, lift: 1.617, conv: 1.387)
{X-Men (2000)} -> {Godfather, The (1972)} (conf: 0.504, supp: 0.110, lift: 1.600, conv: 1.381)
{X-Men (2000)} -> {Incredibles, The (2004)} (conf: 0.504, supp: 0.110, lift: 2.458, conv: 1.602)
{Léon: The Professional (a.k.a. The Professional) (Léon) (1994)} -> {Schindler's List (1993)} (conf: 0.504, supp: 0.110, lift: 1.397, conv: 1.288)
{Léon: The Professional (a.k.a. The Professional) (Léon) (1994)} -> {Sixth Sense, The (1999)} (conf: 0.504, supp: 0.110, lift: 1.717, conv: 1.424)
{Léon: The Professional (a.k.a. The Professional) (Léon) (1994)} -> {True Lies (1994)} (conf: 0.504, supp: 0.110, lift: 1.726, conv: 1.427)
{X-Men (2000)} -> {Memento (2000)} (conf: 0.504, supp: 0.110, lift: 1.933, conv: 1.490)
{One Flew Over the Cuckoo's Nest (1

{Hunt for Red October, The (1990)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.789, supp: 0.116, lift: 1.917, conv: 2.788)
{Hunt for Red October, The (1990)} -> {Star Wars: Episode VI - Return of the Jedi (1983)} (conf: 0.789, supp: 0.116, lift: 2.455, conv: 3.215)
{Get Shorty (1995)} -> {Forrest Gump (1994)} (conf: 0.798, supp: 0.116, lift: 1.479, conv: 2.278)
{Monty Python's Life of Brian (1979)} -> {Monty Python and the Holy Grail (1975)} (conf: 0.798, supp: 0.116, lift: 3.578, conv: 3.842)
{Back to the Future Part III (1990)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.807, supp: 0.116, lift: 1.961, conv: 3.046)
{Inglourious Basterds (2009)} -> {Fight Club (1999)} (conf: 0.807, supp: 0.116, lift: 2.258, conv: 3.327)
{Total Recall (1990)} -> {Men in Black (a.k.a. MIB) (1997)} (conf: 0.807, supp: 0.116, lift: 2.983, conv: 3.776)
{Total Recall (1990)} -> {Star Wars: Episode VI - Return of the Jedi (1983)} (conf: 0.807, supp: 0.116, lift: 2.511, conv: 3.513)
{Ai

{Incredibles, The (2004)} -> {Dark Knight, The (2008)} (conf: 0.584, supp: 0.120, lift: 2.391, conv: 1.817)
{Truman Show, The (1998)} -> {Good Will Hunting (1997)} (conf: 0.584, supp: 0.120, lift: 2.527, conv: 1.848)
{Incredibles, The (2004)} -> {Seven (a.k.a. Se7en) (1995)} (conf: 0.584, supp: 0.120, lift: 1.755, conv: 1.604)
{Incredibles, The (2004)} -> {Sixth Sense, The (1999)} (conf: 0.584, supp: 0.120, lift: 1.990, conv: 1.698)
{Incredibles, The (2004)} -> {Star Wars: Episode VI - Return of the Jedi (1983)} (conf: 0.584, supp: 0.120, lift: 1.818, conv: 1.631)
{Truman Show, The (1998)} -> {Titanic (1997)} (conf: 0.584, supp: 0.120, lift: 2.545, conv: 1.852)
{Blade Runner (1982)} -> {Aliens (1986)} (conf: 0.589, supp: 0.120, lift: 2.850, conv: 1.929)
{Blade Runner (1982)} -> {Reservoir Dogs (1992)} (conf: 0.589, supp: 0.120, lift: 2.741, conv: 1.909)
{Twister (1996)} -> {Braveheart (1995)} (conf: 0.593, supp: 0.120, lift: 1.528, conv: 1.504)
{E.T. the Extra-Terrestrial (1982)} -> {F

{Princess Bride, The (1987)} -> {Indiana Jones and the Last Crusade (1989)} (conf: 0.549, supp: 0.128, lift: 2.393, conv: 1.710)
{Good Will Hunting (1997)} -> {Lord of the Rings: The Two Towers, The (2002)} (conf: 0.553, supp: 0.128, lift: 1.795, conv: 1.548)
{Indiana Jones and the Last Crusade (1989)} -> {Ghostbusters (a.k.a. Ghost Busters) (1984)} (conf: 0.557, supp: 0.128, lift: 2.832, conv: 1.814)
{Indiana Jones and the Last Crusade (1989)} -> {Monty Python and the Holy Grail (1975)} (conf: 0.557, supp: 0.128, lift: 2.499, conv: 1.755)
{Indiana Jones and the Last Crusade (1989)} -> {Princess Bride, The (1987)} (conf: 0.557, supp: 0.128, lift: 2.393, conv: 1.732)
{Stargate (1994)} -> {Lion King, The (1994)} (conf: 0.557, supp: 0.128, lift: 1.976, conv: 1.621)
{Star Wars: Episode I - The Phantom Menace (1999)} -> {Lord of the Rings: The Two Towers, The (2002)} (conf: 0.557, supp: 0.128, lift: 1.808, conv: 1.562)
{Titanic (1997)} -> {Pirates of the Caribbean: The Curse of the Black Pe

{Memento (2000)} -> {Pirates of the Caribbean: The Curse of the Black Pearl (2003)} (conf: 0.535, supp: 0.139, lift: 2.189, conv: 1.624)
{Mask, The (1994)} -> {Die Hard: With a Vengeance (1995)} (conf: 0.541, supp: 0.139, lift: 2.293, conv: 1.666)
{Mask, The (1994)} -> {Dumb & Dumber (Dumb and Dumber) (1994)} (conf: 0.541, supp: 0.139, lift: 2.483, conv: 1.705)
{Dark Knight, The (2008)} -> {Memento (2000)} (conf: 0.570, supp: 0.139, lift: 2.189, conv: 1.721)
{Pirates of the Caribbean: The Curse of the Black Pearl (2003)} -> {Memento (2000)} (conf: 0.570, supp: 0.139, lift: 2.189, conv: 1.721)
{Pirates of the Caribbean: The Curse of the Black Pearl (2003)} -> {Silence of the Lambs, The (1991)} (conf: 0.570, supp: 0.139, lift: 1.247, conv: 1.263)
{Alien (1979)} -> {Die Hard (1988)} (conf: 0.582, supp: 0.139, lift: 2.449, conv: 1.825)
{Alien (1979)} -> {Men in Black (a.k.a. MIB) (1997)} (conf: 0.582, supp: 0.139, lift: 2.152, conv: 1.746)
{Die Hard (1988)} -> {Alien (1979)} (conf: 0.586, 

{Memento (2000)} -> {Seven (a.k.a. Se7en) (1995)} (conf: 0.597, supp: 0.156, lift: 1.795, conv: 1.658)
{Mask, The (1994)} -> {Mrs. Doubtfire (1993)} (conf: 0.605, supp: 0.156, lift: 2.563, conv: 1.934)
{Mask, The (1994)} -> {Speed (1994)} (conf: 0.605, supp: 0.156, lift: 2.159, conv: 1.822)
{Pirates of the Caribbean: The Curse of the Black Pearl (2003)} -> {Incredibles, The (2004)} (conf: 0.638, supp: 0.156, lift: 3.111, conv: 2.194)
{Alien (1979)} -> {American Beauty (1999)} (conf: 0.651, supp: 0.156, lift: 1.946, conv: 1.905)
{Alien (1979)} -> {Pulp Fiction (1994)} (conf: 0.651, supp: 0.156, lift: 1.293, conv: 1.422)
{Beauty and the Beast (1991)} -> {Batman (1989)} (conf: 0.651, supp: 0.156, lift: 2.100, conv: 1.976)
{Die Hard (1988)} -> {Saving Private Ryan (1998)} (conf: 0.655, supp: 0.156, lift: 2.126, conv: 2.006)
{Mrs. Doubtfire (1993)} -> {Aladdin (1992)} (conf: 0.660, supp: 0.156, lift: 2.199, conv: 2.057)
{Mrs. Doubtfire (1993)} -> {Mask, The (1994)} (conf: 0.660, supp: 0.156

{Saving Private Ryan (1998)} -> {Pulp Fiction (1994)} (conf: 0.676, supp: 0.208, lift: 1.342, conv: 1.531)
{Aladdin (1992)} -> {Lion King, The (1994)} (conf: 0.694, supp: 0.208, lift: 2.461, conv: 2.346)
{Lion King, The (1994)} -> {Aladdin (1992)} (conf: 0.738, supp: 0.208, lift: 2.461, conv: 2.676)
{Ace Ventura: Pet Detective (1994)} -> {Forrest Gump (1994)} (conf: 0.789, supp: 0.208, lift: 1.463, conv: 2.181)
{Mask, The (1994)} -> {Forrest Gump (1994)} (conf: 0.809, supp: 0.208, lift: 1.500, conv: 2.411)
{Star Wars: Episode IV - A New Hope (1977)} -> {Back to the Future (1985)} (conf: 0.510, supp: 0.210, lift: 1.819, conv: 1.469)
{Jurassic Park (1993)} -> {Lion King, The (1994)} (conf: 0.538, supp: 0.210, lift: 1.907, conv: 1.554)
{Jurassic Park (1993)} -> {Star Wars: Episode VI - Return of the Jedi (1983)} (conf: 0.538, supp: 0.210, lift: 1.674, conv: 1.468)
{Jurassic Park (1993)} -> {True Lies (1994)} (conf: 0.538, supp: 0.210, lift: 1.843, conv: 1.532)
{Braveheart (1995)} -> {Inde

In [48]:
rules=sorted(rules, key=lambda rule: rule.conviction)
for rule in rules:
  print(rule)

{Rock, The (1996)} -> {Shawshank Redemption, The (1994)} (conf: 0.545, supp: 0.108, lift: 1.050, conv: 1.057)
{Austin Powers: The Spy Who Shagged Me (1999)} -> {Shawshank Redemption, The (1994)} (conf: 0.554, supp: 0.110, lift: 1.066, conv: 1.076)
{Babe (1995)} -> {Matrix, The (1999)} (conf: 0.500, supp: 0.105, lift: 1.097, conv: 1.089)
{Pretty Woman (1990)} -> {Matrix, The (1999)} (conf: 0.504, supp: 0.111, lift: 1.105, conv: 1.097)
{Willy Wonka & the Chocolate Factory (1971)} -> {Shawshank Redemption, The (1994)} (conf: 0.563, supp: 0.110, lift: 1.083, conv: 1.099)
{Fugitive, The (1993)} -> {Matrix, The (1999)} (conf: 0.511, supp: 0.159, lift: 1.120, conv: 1.112)
{Dumb & Dumber (Dumb and Dumber) (1994)} -> {Matrix, The (1999)} (conf: 0.511, supp: 0.111, lift: 1.122, conv: 1.114)
{GoldenEye (1995)} -> {Matrix, The (1999)} (conf: 0.523, supp: 0.113, lift: 1.147, conv: 1.140)
{Twister (1996)} -> {Shawshank Redemption, The (1994)} (conf: 0.585, supp: 0.118, lift: 1.126, conv: 1.158)
{Mrs

{Incredibles, The (2004)} -> {Silence of the Lambs, The (1991)} (conf: 0.624, supp: 0.128, lift: 1.364, conv: 1.443)
{Toy Story (1995)} -> {Pulp Fiction (1994)} (conf: 0.656, supp: 0.231, lift: 1.303, conv: 1.443)
{Willy Wonka & the Chocolate Factory (1971)} -> {Seven (a.k.a. Se7en) (1995)} (conf: 0.538, supp: 0.105, lift: 1.616, conv: 1.444)
{Clockwork Orange, A (1971)} -> {Twelve Monkeys (a.k.a. 12 Monkeys) (1995)} (conf: 0.508, supp: 0.100, lift: 1.752, conv: 1.444)
{Seven (a.k.a. Se7en) (1995)} -> {Independence Day (a.k.a. ID4) (1996)} (conf: 0.537, supp: 0.179, lift: 1.621, conv: 1.444)
{Saving Private Ryan (1998)} -> {Independence Day (a.k.a. ID4) (1996)} (conf: 0.537, supp: 0.166, lift: 1.622, conv: 1.445)
{Star Wars: Episode IV - A New Hope (1977)} -> {Godfather, The (1972)} (conf: 0.526, supp: 0.216, lift: 1.671, conv: 1.445)
{Pirates of the Caribbean: The Curse of the Black Pearl (2003)} -> {Lion King, The (1994)} (conf: 0.503, supp: 0.123, lift: 1.785, conv: 1.446)
{Apollo 1

{Beauty and the Beast (1991)} -> {Die Hard: With a Vengeance (1995)} (conf: 0.507, supp: 0.121, lift: 2.147, conv: 1.549)
{Beautiful Mind, A (2001)} -> {Dark Knight, The (2008)} (conf: 0.512, supp: 0.103, lift: 2.097, conv: 1.549)
{American History X (1998)} -> {Good Will Hunting (1997)} (conf: 0.504, supp: 0.107, lift: 2.180, conv: 1.550)
{Godfather: Part II, The (1974)} -> {Good Will Hunting (1997)} (conf: 0.504, supp: 0.107, lift: 2.180, conv: 1.550)
{Austin Powers: The Spy Who Shagged Me (1999)} -> {Saving Private Ryan (1998)} (conf: 0.554, supp: 0.110, lift: 1.797, conv: 1.550)
{American Pie (1999)} -> {Pulp Fiction (1994)} (conf: 0.680, supp: 0.115, lift: 1.350, conv: 1.550)
{Austin Powers: The Spy Who Shagged Me (1999)} -> {Finding Nemo (2003)} (conf: 0.504, supp: 0.100, lift: 2.181, conv: 1.551)
{Mrs. Doubtfire (1993)} -> {Batman Forever (1995)} (conf: 0.500, supp: 0.118, lift: 2.226, conv: 1.551)
{Fargo (1996)} -> {Fight Club (1999)} (conf: 0.586, supp: 0.174, lift: 1.639, con

{Dances with Wolves (1990)} -> {Shawshank Redemption, The (1994)} (conf: 0.707, supp: 0.190, lift: 1.361, conv: 1.641)
{Big Lebowski, The (1998)} -> {Lord of the Rings: The Return of the King, The (2003)} (conf: 0.575, supp: 0.100, lift: 1.898, conv: 1.641)
{Full Metal Jacket (1987)} -> {Braveheart (1995)} (conf: 0.627, supp: 0.105, lift: 1.615, conv: 1.641)
{Beauty and the Beast (1991)} -> {Speed (1994)} (conf: 0.562, supp: 0.134, lift: 2.004, conv: 1.642)
{Goodfellas (1990)} -> {One Flew Over the Cuckoo's Nest (1975)} (conf: 0.524, supp: 0.108, lift: 2.402, conv: 1.642)
{Goodfellas (1990)} -> {Ferris Bueller's Day Off (1986)} (conf: 0.500, supp: 0.103, lift: 2.798, conv: 1.643)
{Pretty Woman (1990)} -> {Terminator 2: Judgment Day (1991)} (conf: 0.615, supp: 0.136, lift: 1.674, conv: 1.643)
{Rock, The (1996)} -> {Speed (1994)} (conf: 0.562, supp: 0.111, lift: 2.005, conv: 1.643)
{Twister (1996)} -> {True Lies (1994)} (conf: 0.569, supp: 0.115, lift: 1.950, conv: 1.644)
{Apollo 13 (199

{Groundhog Day (1993)} -> {Ferris Bueller's Day Off (1986)} (conf: 0.524, supp: 0.123, lift: 2.935, conv: 1.727)
{Fight Club (1999)} -> {American Beauty (1999)} (conf: 0.615, supp: 0.220, lift: 1.838, conv: 1.727)
{Firm, The (1993)} -> {Terminator 2: Judgment Day (1991)} (conf: 0.634, supp: 0.105, lift: 1.726, conv: 1.727)
{Dumb & Dumber (Dumb and Dumber) (1994)} -> {Fugitive, The (1993)} (conf: 0.602, supp: 0.131, lift: 1.931, conv: 1.728)
{Austin Powers: The Spy Who Shagged Me (1999)} -> {Fight Club (1999)} (conf: 0.628, supp: 0.125, lift: 1.758, conv: 1.728)
{Ghostbusters (a.k.a. Ghost Busters) (1984)} -> {Mission: Impossible (1996)} (conf: 0.575, supp: 0.113, lift: 2.165, conv: 1.728)
{Back to the Future Part II (1989)} -> {Pulp Fiction (1994)} (conf: 0.713, supp: 0.102, lift: 1.416, conv: 1.729)
{Ocean's Eleven (2001)} -> {Matrix Reloaded, The (2003)} (conf: 0.513, supp: 0.100, lift: 3.257, conv: 1.729)
{Batman Begins (2005)} -> {Ocean's Eleven (2001)} (conf: 0.534, supp: 0.102, l

{Blade Runner (1982)} -> {Lord of the Rings: The Fellowship of the Ring, The (2001)} (conf: 0.637, supp: 0.130, lift: 1.963, conv: 1.861)
{Back to the Future (1985)} -> {Matrix, The (1999)} (conf: 0.708, supp: 0.198, lift: 1.553, conv: 1.861)
{Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)} -> {Forrest Gump (1994)} (conf: 0.753, supp: 0.120, lift: 1.395, conv: 1.862)
{L.A. Confidential (1997)} -> {Forrest Gump (1994)} (conf: 0.753, supp: 0.120, lift: 1.395, conv: 1.862)
{Trainspotting (1996)} -> {Lord of the Rings: The Fellowship of the Ring, The (2001)} (conf: 0.637, supp: 0.107, lift: 1.963, conv: 1.862)
{Green Mile, The (1999)} -> {Gladiator (2000)} (conf: 0.613, supp: 0.111, lift: 2.198, conv: 1.862)
{Minority Report (2002)} -> {Men in Black (a.k.a. MIB) (1997)} (conf: 0.608, supp: 0.120, lift: 2.249, conv: 1.863)
{Toy Story 2 (1999)} -> {Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)} (conf: 0.639, supp: 0.102, lift: 1.9

{Minority Report (2002)} -> {Ocean's Eleven (2001)} (conf: 0.608, supp: 0.120, lift: 3.118, conv: 2.055)
{Matrix Reloaded, The (2003)} -> {Sixth Sense, The (1999)} (conf: 0.656, supp: 0.103, lift: 2.236, conv: 2.055)
{Rain Man (1988)} -> {Sixth Sense, The (1999)} (conf: 0.656, supp: 0.103, lift: 2.236, conv: 2.055)
{Full Metal Jacket (1987)} -> {Godfather, The (1972)} (conf: 0.667, supp: 0.111, lift: 2.118, conv: 2.056)
{American Pie (1999)} -> {Star Wars: Episode VI - Return of the Jedi (1983)} (conf: 0.670, supp: 0.113, lift: 2.085, conv: 2.056)
{Lord of the Rings: The Return of the King, The (2003)} -> {Pirates of the Caribbean: The Curse of the Black Pearl (2003)} (conf: 0.632, supp: 0.192, lift: 2.589, conv: 2.056)
{Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)} -> {Lord of the Rings: The Two Towers, The (2002)} (conf: 0.664, supp: 0.120, lift: 2.153, conv: 2.057)
{Reservoir Dogs (1992)} -> {Sixth Sense, The (1999)} (conf: 0.656, supp: 0.141, lift: 2.237, conv: 2.057)
{I

{Stand by Me (1986)} -> {American Beauty (1999)} (conf: 0.692, supp: 0.103, lift: 2.070, conv: 2.163)
{Pretty Woman (1990)} -> {Pulp Fiction (1994)} (conf: 0.770, supp: 0.170, lift: 1.531, conv: 2.163)
{Clockwork Orange, A (1971)} -> {Fargo (1996)} (conf: 0.675, supp: 0.133, lift: 2.275, conv: 2.164)
{Matrix Reloaded, The (2003)} -> {Gladiator (2000)} (conf: 0.667, supp: 0.105, lift: 2.392, conv: 2.164)
{Wizard of Oz, The (1939)} -> {Men in Black (a.k.a. MIB) (1997)} (conf: 0.663, supp: 0.100, lift: 2.451, conv: 2.165)
{Dumb & Dumber (Dumb and Dumber) (1994)} -> {Aladdin (1992)} (conf: 0.677, supp: 0.148, lift: 2.256, conv: 2.165)
{Rain Man (1988)} -> {Star Wars: Episode V - The Empire Strikes Back (1980)} (conf: 0.698, supp: 0.110, lift: 2.018, conv: 2.165)
{Interview with the Vampire: The Vampire Chronicles (1994)} -> {Pulp Fiction (1994)} (conf: 0.771, supp: 0.138, lift: 1.531, conv: 2.166)
{Die Hard (1988)} -> {Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark)

{Interview with the Vampire: The Vampire Chronicles (1994)} -> {Jurassic Park (1993)} (conf: 0.752, supp: 0.134, lift: 1.928, conv: 2.462)
{Breakfast Club, The (1985)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.761, supp: 0.141, lift: 1.850, conv: 2.463)
{V for Vendetta (2006)} -> {Memento (2000)} (conf: 0.700, supp: 0.115, lift: 2.686, conv: 2.464)
{Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)} -> {Forrest Gump (1994)} (conf: 0.813, supp: 0.143, lift: 1.508, conv: 2.465)
{Nightmare Before Christmas, The (1993)} -> {Jurassic Park (1993)} (conf: 0.753, supp: 0.115, lift: 1.929, conv: 2.466)
{Meet the Parents (2000)} -> {Forrest Gump (1994)} (conf: 0.813, supp: 0.121, lift: 1.508, conv: 2.466)
{Ferris Bueller's Day Off (1986)} -> {Star Wars: Episode VI - Return of the Jedi (1983)} (conf: 0.725, supp: 0.130, lift: 2.256, conv: 2.466)
{Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001

{Starship Troopers (1997)} -> {Star Wars: Episode V - The Empire Strikes Back (1980)} (conf: 0.800, supp: 0.105, lift: 2.313, conv: 3.270)
{Harry Potter and the Goblet of Fire (2005)} -> {Forrest Gump (1994)} (conf: 0.859, supp: 0.100, lift: 1.593, conv: 3.271)
{Interview with the Vampire: The Vampire Chronicles (1994)} -> {Batman (1989)} (conf: 0.789, supp: 0.141, lift: 2.546, conv: 3.271)
{Big (1988)} -> {Back to the Future (1985)} (conf: 0.780, supp: 0.116, lift: 2.783, conv: 3.275)
{Trainspotting (1996)} -> {Fight Club (1999)} (conf: 0.804, supp: 0.134, lift: 2.250, conv: 3.277)
{Being John Malkovich (1999)} -> {Pulp Fiction (1994)} (conf: 0.848, supp: 0.138, lift: 1.686, conv: 3.278)
{Cast Away (2000)} -> {Shrek (2001)} (conf: 0.780, supp: 0.128, lift: 2.799, conv: 3.279)
{Mission: Impossible (1996)} -> {Independence Day (a.k.a. ID4) (1996)} (conf: 0.796, supp: 0.211, lift: 2.405, conv: 3.283)
{Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone)

In [49]:
rules=sorted(rules, key=lambda rule: rule.lift)
for rule in rules:
  print(rule)

{Rock, The (1996)} -> {Shawshank Redemption, The (1994)} (conf: 0.545, supp: 0.108, lift: 1.050, conv: 1.057)
{Austin Powers: The Spy Who Shagged Me (1999)} -> {Shawshank Redemption, The (1994)} (conf: 0.554, supp: 0.110, lift: 1.066, conv: 1.076)
{Willy Wonka & the Chocolate Factory (1971)} -> {Shawshank Redemption, The (1994)} (conf: 0.563, supp: 0.110, lift: 1.083, conv: 1.099)
{Babe (1995)} -> {Matrix, The (1999)} (conf: 0.500, supp: 0.105, lift: 1.097, conv: 1.089)
{Pretty Woman (1990)} -> {Matrix, The (1999)} (conf: 0.504, supp: 0.111, lift: 1.105, conv: 1.097)
{Fugitive, The (1993)} -> {Matrix, The (1999)} (conf: 0.511, supp: 0.159, lift: 1.120, conv: 1.112)
{Dumb & Dumber (Dumb and Dumber) (1994)} -> {Matrix, The (1999)} (conf: 0.511, supp: 0.111, lift: 1.122, conv: 1.114)
{Twister (1996)} -> {Shawshank Redemption, The (1994)} (conf: 0.585, supp: 0.118, lift: 1.126, conv: 1.158)
{Willy Wonka & the Chocolate Factory (1971)} -> {Forrest Gump (1994)} (conf: 0.613, supp: 0.120, lif

{Clerks (1994)} -> {Pulp Fiction (1994)} (conf: 0.808, supp: 0.138, lift: 1.605, conv: 2.583)
{Apocalypse Now (1979)} -> {Jurassic Park (1993)} (conf: 0.626, supp: 0.110, lift: 1.605, conv: 1.631)
{Speed (1994)} -> {Schindler's List (1993)} (conf: 0.579, supp: 0.162, lift: 1.605, conv: 1.518)
{Donnie Darko (2001)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.661, supp: 0.118, lift: 1.605, conv: 1.734)
{Goodfellas (1990)} -> {Independence Day (a.k.a. ID4) (1996)} (conf: 0.532, supp: 0.110, lift: 1.606, conv: 1.428)
{Toy Story 2 (1999)} -> {Matrix, The (1999)} (conf: 0.732, supp: 0.116, lift: 1.606, conv: 2.031)
{Waterworld (1995)} -> {Pulp Fiction (1994)} (conf: 0.809, supp: 0.152, lift: 1.607, conv: 2.596)
{Gladiator (2000)} -> {Seven (a.k.a. Se7en) (1995)} (conf: 0.535, supp: 0.149, lift: 1.609, conv: 1.436)
{Die Hard (1988)} -> {Usual Suspects, The (1995)} (conf: 0.538, supp: 0.128, lift: 1.609, conv: 1.440)
{Clerks (1994)} -> {Braveheart (1995)} (conf: 0.625, supp: 0.107,

{American Pie (1999)} -> {Star Wars: Episode IV - A New Hope (1977)} (conf: 0.709, supp: 0.120, lift: 1.722, conv: 2.021)
{Groundhog Day (1993)} -> {Seven (a.k.a. Se7en) (1995)} (conf: 0.573, supp: 0.134, lift: 1.723, conv: 1.564)
{Babe (1995)} -> {Twelve Monkeys (a.k.a. 12 Monkeys) (1995)} (conf: 0.500, supp: 0.105, lift: 1.723, conv: 1.420)
{GoldenEye (1995)} -> {Twelve Monkeys (a.k.a. 12 Monkeys) (1995)} (conf: 0.500, supp: 0.108, lift: 1.723, conv: 1.420)
{Matrix, The (1999)} -> {Star Wars: Episode VI - Return of the Jedi (1983)} (conf: 0.554, supp: 0.252, lift: 1.724, conv: 1.522)
{Star Wars: Episode VI - Return of the Jedi (1983)} -> {Matrix, The (1999)} (conf: 0.786, supp: 0.252, lift: 1.724, conv: 2.540)
{Ghost (1990)} -> {Seven (a.k.a. Se7en) (1995)} (conf: 0.574, supp: 0.108, lift: 1.725, conv: 1.566)
{Groundhog Day (1993)} -> {Aladdin (1992)} (conf: 0.517, supp: 0.121, lift: 1.725, conv: 1.451)
{Star Wars: Episode V - The Empire Strikes Back (1980)} -> {Fargo (1996)} (conf: 

{Speed (1994)} -> {Independence Day (a.k.a. ID4) (1996)} (conf: 0.614, supp: 0.172, lift: 1.854, conv: 1.733)
{Dark Knight, The (2008)} -> {Shrek (2001)} (conf: 0.517, supp: 0.126, lift: 1.854, conv: 1.493)
{Truman Show, The (1998)} -> {Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)} (conf: 0.608, supp: 0.125, lift: 1.854, conv: 1.715)
{Home Alone (1990)} -> {Terminator 2: Judgment Day (1991)} (conf: 0.681, supp: 0.130, lift: 1.855, conv: 1.984)
{Clockwork Orange, A (1971)} -> {Star Wars: Episode V - The Empire Strikes Back (1980)} (conf: 0.642, supp: 0.126, lift: 1.855, conv: 1.825)
{Kill Bill: Vol. 2 (2004)} -> {Matrix, The (1999)} (conf: 0.845, supp: 0.152, lift: 1.855, conv: 3.522)
{Braveheart (1995)} -> {Fugitive, The (1993)} (conf: 0.578, supp: 0.225, lift: 1.856, conv: 1.632)
{Fugitive, The (1993)} -> {Braveheart (1995)} (conf: 0.721, supp: 0.225, lift: 1.856, conv: 2.192)
{Airplane! (1980)} -> {Jurassic Park (1993)} (conf: 0.724, supp: 0.103, lif

{When Harry Met Sally... (1989)} -> {Toy Story (1995)} (conf: 0.729, supp: 0.102, lift: 2.069, conv: 2.393)
{Eternal Sunshine of the Spotless Mind (2004)} -> {Lord of the Rings: The Fellowship of the Ring, The (2001)} (conf: 0.672, supp: 0.144, lift: 2.070, conv: 2.058)
{Star Wars: Episode I - The Phantom Menace (1999)} -> {Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)} (conf: 0.679, supp: 0.156, lift: 2.070, conv: 2.091)
{Fifth Element, The (1997)} -> {Lord of the Rings: The Two Towers, The (2002)} (conf: 0.638, supp: 0.121, lift: 2.070, conv: 1.911)
{Stand by Me (1986)} -> {American Beauty (1999)} (conf: 0.692, supp: 0.103, lift: 2.070, conv: 2.163)
{2001: A Space Odyssey (1968)} -> {Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)} (conf: 0.679, supp: 0.121, lift: 2.071, conv: 2.093)
{Ferris Bueller's Day Off (1986)} -> {Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)} (conf: 0.679, supp: 0

{Airplane! (1980)} -> {Star Wars: Episode V - The Empire Strikes Back (1980)} (conf: 0.747, supp: 0.107, lift: 2.160, conv: 2.587)
{Star Trek: Generations (1994)} -> {Aladdin (1992)} (conf: 0.648, supp: 0.115, lift: 2.160, conv: 1.989)
{Dogma (1999)} -> {Fight Club (1999)} (conf: 0.772, supp: 0.100, lift: 2.161, conv: 2.820)
{L.A. Confidential (1997)} -> {Godfather, The (1972)} (conf: 0.680, supp: 0.108, lift: 2.162, conv: 2.144)
{Goodfellas (1990)} -> {Memento (2000)} (conf: 0.563, supp: 0.116, lift: 2.162, conv: 1.694)
{Four Weddings and a Funeral (1994)} -> {Batman (1989)} (conf: 0.670, supp: 0.113, lift: 2.162, conv: 2.091)
{Trainspotting (1996)} -> {Twelve Monkeys (a.k.a. 12 Monkeys) (1995)} (conf: 0.627, supp: 0.105, lift: 2.162, conv: 1.905)
{Goodfellas (1990)} -> {Good Will Hunting (1997)} (conf: 0.500, supp: 0.103, lift: 2.163, conv: 1.538)
{Monty Python and the Holy Grail (1975)} -> {Finding Nemo (2003)} (conf: 0.500, supp: 0.111, lift: 2.163, conv: 1.538)
{E.T. the Extra-Ter

{City of God (Cidade de Deus) (2002)} -> {Fight Club (1999)} (conf: 0.853, supp: 0.105, lift: 2.388, conv: 4.382)
{Fifth Element, The (1997)} -> {Star Wars: Episode VI - Return of the Jedi (1983)} (conf: 0.767, supp: 0.146, lift: 2.388, conv: 2.916)
{Truman Show, The (1998)} -> {Finding Nemo (2003)} (conf: 0.552, supp: 0.113, lift: 2.388, conv: 1.716)
{Star Wars: Episode II - Attack of the Clones (2002)} -> {Star Wars: Episode V - The Empire Strikes Back (1980)} (conf: 0.826, supp: 0.125, lift: 2.388, conv: 3.761)
{Mars Attacks! (1996)} -> {Star Wars: Episode VI - Return of the Jedi (1983)} (conf: 0.767, supp: 0.108, lift: 2.388, conv: 2.918)
{Ghost (1990)} -> {Speed (1994)} (conf: 0.670, supp: 0.126, lift: 2.389, conv: 2.178)
{Toy Story 2 (1999)} -> {Sixth Sense, The (1999)} (conf: 0.701, supp: 0.111, lift: 2.389, conv: 2.363)
{Star Wars: Episode V - The Empire Strikes Back (1980)} -> {Star Wars: Episode VI - Return of the Jedi (1983)} (conf: 0.768, supp: 0.266, lift: 2.389, conv: 2.9

{Star Wars: Episode III - Revenge of the Sith (2005)} -> {Shrek (2001)} (conf: 0.795, supp: 0.102, lift: 2.852, conv: 3.516)
{Pirates of the Caribbean: The Curse of the Black Pearl (2003)} -> {Monsters, Inc. (2001)} (conf: 0.617, supp: 0.151, lift: 2.853, conv: 2.048)
{Monsters, Inc. (2001)} -> {Pirates of the Caribbean: The Curse of the Black Pearl (2003)} (conf: 0.697, supp: 0.151, lift: 2.853, conv: 2.494)
{Harry Potter and the Prisoner of Azkaban (2004)} -> {Shrek (2001)} (conf: 0.796, supp: 0.121, lift: 2.855, conv: 3.531)
{Prestige, The (2006)} -> {Memento (2000)} (conf: 0.744, supp: 0.110, lift: 2.856, conv: 2.893)
{GoldenEye (1995)} -> {Clear and Present Danger (1994)} (conf: 0.515, supp: 0.111, lift: 2.857, conv: 1.691)
{Monsters, Inc. (2001)} -> {Kill Bill: Vol. 2 (2004)} (conf: 0.515, supp: 0.111, lift: 2.857, conv: 1.691)
{Clear and Present Danger (1994)} -> {GoldenEye (1995)} (conf: 0.618, supp: 0.111, lift: 2.857, conv: 2.052)
{Kill Bill: Vol. 2 (2004)} -> {Monsters, Inc.

{American Pie (1999)} -> {There's Something About Mary (1998)} (conf: 0.612, supp: 0.103, lift: 3.553, conv: 2.132)
{Batman Begins (2005)} -> {Matrix Reloaded, The (2003)} (conf: 0.560, supp: 0.107, lift: 3.561, conv: 1.917)
{Matrix Reloaded, The (2003)} -> {Batman Begins (2005)} (conf: 0.677, supp: 0.107, lift: 3.561, conv: 2.508)
{Monsters, Inc. (2001)} -> {Shrek 2 (2004)} (conf: 0.538, supp: 0.116, lift: 3.566, conv: 1.838)
{Shrek 2 (2004)} -> {Monsters, Inc. (2001)} (conf: 0.772, supp: 0.116, lift: 3.566, conv: 3.433)
{Clear and Present Danger (1994)} -> {Outbreak (1995)} (conf: 0.591, supp: 0.107, lift: 3.569, conv: 2.040)
{Outbreak (1995)} -> {Clear and Present Danger (1994)} (conf: 0.644, supp: 0.107, lift: 3.569, conv: 2.300)
{Dark Knight, The (2008)} -> {Iron Man (2008)} (conf: 0.550, supp: 0.134, lift: 3.571, conv: 1.881)
{Iron Man (2008)} -> {Dark Knight, The (2008)} (conf: 0.872, supp: 0.134, lift: 3.571, conv: 5.920)
{Ferris Bueller's Day Off (1986)} -> {Indiana Jones and 


#Analizando el Set de Datos de Movielens:

En principio se seleccionó que el soporte mínimo sea de 0.1 para que estén en por lo menos el 10 % de las transacciones y que no genere demasiadas relaciones improbables y con una confianza mínima del 0.5 y con una longitud máxima de 2.
Cada métrica permite observar en mayor profundidad ciertos patrones de los espectadores: 

-Con mayor índice de convicción, con lo que hay alta implicación, los casos donde la convicción es más alta son generalmente películas que pertenecen están directamente relacionadas siendo parte una trilogía o saga, como Star Wars en episodio no consecutivos (III y V) con convicción de 5.8  y llegando a peliculas directamente relacionados como Kill Bill y El padrino, con convicciones mayores a 10.

-Algo similar se observa en menor medida en la métrica de confianza, aunque el par de películas de El padrino (1972) y El padrino II (1974) son el par con mayor confianza, la relaciones con alta confianza se dan por pertenecer a géneros similares (por ejemplo Ciencia Ficción: {RoboCop (1987)} -> {Terminator, The (1984)} conf: 0.871 , o por ser películas  populares del mismo año ( para el año 1994:, Four Weddings and a Funeral (1994)} -> {Forrest Gump (1994)} conf: 0.922 ; para el año   1999 (Mummy, The (1999)} -> {Matrix, The (1999)} conf: 0.874; y el año Cast Away (2000)} -> {Gladiator (2000)} conf: 0.710. Otra relación es que pertenezcan al mismo director {Dark Knight Rises, The (2012)}- >{Inception (2010)} conf: 0.882; ambas dirigidas por Christopher Nolan) .

-Para soporte puede destacarse el año 1994 con: {Shawshank Redemption, The (1994)} -> {Forrest Gump (1994)} supp: 0.379 ,{Pulp Fiction (1994)} -> {Forrest Gump (1994)} supp: 0.377 ,{Shawshank Redemption, The (1994)} -> {Pulp Fiction (1994)}, supp: 0.364
De las tres, la más popular en ese año fue Forrest Gump, las otras dos ganaron popularidad y “prestigio” en los años subsiguientes.

-Con la métrica de Lift también se ven valores altos para películas que previamente he mencionado, aunque ya empiezan a notarse relaciones distintas porque a diferencia de la métrica de confianza, lift no tiene dirección, sino que valores superiores a 1 muestran una relación no azarosa y positiva entre A y B, como películas del mismo estudio de animación {Finding Nemo (2003)} -> {Incredibles, The (2004)} lift: 3.426; O de mismo año pero de temática no relacionadas {Departed, The (2006)} -> {V for Vendetta (2006)} (lift: 3.535)
